In [ ]:
# [1] Dọn dẹp môi trường cũ
!pip uninstall -y opencv-python opencv-python-headless numpy torch tensorflow > /dev/null 2>&1
!rm -rf ~/.cache/pip

# [2] Cài đặt phiên bản chuẩn (dùng --no-deps để tránh xung đột)
!pip install --no-deps -q numpy==1.24.4
!pip install --no-deps -q opencv-python-headless==4.9.0.80
!pip install --no-deps -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install --no-deps -q tensorflow==2.12.0  # Phiên bản ổn định với Colab
!pip install --no-deps -q protobuf==3.20.3  # Tránh xung đột với TensorFlow
!pip install --no-deps -q transformers==4.30.0
!pip install --no-deps -q protobuf==3.20.3

# [3] Chủ động khởi động lại bằng Python
from google.colab import runtime
runtime.unassign()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 651.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.5 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
YOLO_CFG = "/content/drive/MyDrive/Colab Notebooks/Final/yolov3-tiny.cfg"
YOLO_WEIGHTS = "/content/drive/MyDrive/Colab Notebooks/Final/yolov3-tiny.weights"
VIT_MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/Final/best_vit_model.pth"
MOVENET_PATH = "/content/drive/MyDrive/Colab Notebooks/Final/movenet_lightning.tflite"


In [10]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
import tensorflow as tf
import warnings
from IPython.display import display, Javascript, HTML
import ipywidgets as widgets
import time
import base64
import threading
import os
import traceback
from transformers import ViTConfig, ViTForImageClassification

warnings.filterwarnings('ignore')

# ==================== CẤU HÌNH ====================
YOLO_CFG = "/content/drive/MyDrive/Colab Notebooks/Final/yolov3-tiny.cfg"
YOLO_WEIGHTS = "/content/drive/MyDrive/Colab Notebooks/Final/yolov3-tiny.weights"
VIT_MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/Final/best_vit_model.pth"
MOVENET_PATH = "/content/drive/MyDrive/Colab Notebooks/Final/movenet_lightning.tflite"

CLASS_NAMES = [
    "calling", "clapping", "cycling", "dancing", "drinking",
    "eating", "fighting", "hugging", "laughing", "listening_to_music",
    "running", "sitting", "sleeping", "texting", "using_laptop"
]

SKELETON_CONNECTIONS = [
    # Đầu và cổ
    (5, 6),    # Tai trái - Tai phải
    (5, 0),    # Tai trái - Mũi
    (6, 0),    # Tai phải - Mũi
    (0, 1),    # Mũi - Mắt trái
    (0, 2),    # Mũi - Mắt phải

    # Thân trên
    (5, 7),    # Tai trái - Vai trái
    (6, 8),    # Tai phải - Vai phải
    (7, 9),    # Vai trái - Khuỷu tay trái
    (8, 10),   # Vai phải - Khuỷu tay phải
    (7, 11),   # Vai trái - Hông trái
    (8, 12),   # Vai phải - Hông phải
    (11, 12),  # Hông trái - Hông phải

    # Thân dưới
    (11, 13),  # Hông trái - Đầu gối trái
    (12, 14),  # Hông phải - Đầu gối phải
    (13, 15),  # Đầu gối trái - Mắt cá trái
    (14, 16),  # Đầu gối phải - Mắt cá phải

    # Tay
    (9, 11),   # Khuỷu tay trái - Cổ tay trái
    (10, 12),  # Khuỷu tay phải - Cổ tay phải
]

# ==================== KHỞI TẠO MODEL ====================
def init_models():
    print("🔧 Đang khởi tạo model...")
    models = {}

    try:
        # YOLO
        net = cv2.dnn.readNetFromDarknet(YOLO_CFG, YOLO_WEIGHTS)
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        ln = net.getLayerNames()
        ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
        models['yolo'] = (net, ln)
        print("✅ YOLO loaded")

        # MoveNet
        interpreter = tf.lite.Interpreter(model_path=MOVENET_PATH)
        interpreter.allocate_tensors()
        models['movenet'] = interpreter
        print("✅ MoveNet loaded")

    except Exception as e:
        print(f"❌ Lỗi khởi tạo model: {e}")
        raise

    return models

# ==================== CAMERA ====================
class VideoStream:
    def __init__(self):
        self.is_active = False
        self.frame = None
        self.lock = threading.Lock()
        self.resolution = (640, 480)
        self.output_handle = None
        self.last_update = 0

    def start(self):
        if not self.is_active:
            self.is_active = True
            # Tạo output widget để hiển thị
            self.output_handle = display(display_id='video_output')
            print(f"📷 Camera đã bật ({self.resolution[0]}x{self.resolution[1]})")

            js = Javascript(f"""
            const video = document.createElement('video');
            video.width = {self.resolution[0]};
            video.height = {self.resolution[1]};
            video.style.transform = 'scaleX(-1)';
            document.body.appendChild(video);

            let lastFrameTime = 0;

            const processFrame = async () => {{
                if (!window.cameraActive) return;

                // Giới hạn tốc độ frame (~20fps)
                const now = Date.now();
                if (now - lastFrameTime < 50) {{
                    requestAnimationFrame(processFrame);
                    return;
                }}
                lastFrameTime = now;

                try {{
                    const canvas = document.createElement('canvas');
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    const ctx = canvas.getContext('2d');
                    ctx.drawImage(video, 0, 0);

                    const imgData = canvas.toDataURL('image/jpeg', 0.8);
                    if (imgData && imgData.length > 1000) {{
                        google.colab.kernel.invokeFunction('notebook.processFrame', [imgData], {{}});
                    }}
                }} catch (e) {{
                    console.log(e);
                }}

                requestAnimationFrame(processFrame);
            }};

            navigator.mediaDevices.getUserMedia({{
                video: {{
                    width: {{ideal: {self.resolution[0]}}},
                    height: {{ideal: {self.resolution[1]}}},
                    facingMode: 'user',
                    frameRate: {{ideal: 30}}
                }}
            }}).then(stream => {{
                window.cameraActive = true;
                video.srcObject = stream;
                video.play();
                processFrame();
            }});
            """)
            display(js)

    def stop(self):
        if self.is_active:
            self.is_active = False
            js = Javascript("""
            window.cameraActive = false;
            const video = document.querySelector('video');
            if (video && video.srcObject) {
                video.srcObject.getTracks().forEach(track => track.stop());
                video.remove();
            }
            """)
            display(js)
            if self.output_handle:
                self.output_handle.update(HTML(""))
            print("🛑 Camera đã tắt")

# ==================== ỨNG DỤNG CHÍNH ====================
class ActionRecognitionApp:
    def __init__(self):
        # Khởi tạo thuộc tính
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"⚙️ Thiết bị: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

        # Khởi tạo các model
        self.models = init_models()
        self.stream = VideoStream()
        self.results = []
        self.fps = 0
        self.last_time = time.time()
        self.vit_ready = False

        # Setup UI
        self.setup_ui()

        # Load ViT model
        self.load_vit_model()

    def setup_ui(self):
        """Giao diện người dùng"""
        self.btn_start = widgets.Button(
            description="BẬT CAMERA",
            button_style='success',
            icon='video',
            layout=widgets.Layout(width='150px', height='45px'),
            disabled=True
        )
        self.btn_stop = widgets.Button(
            description="TẮT CAMERA",
            button_style='danger',
            icon='stop',
            layout=widgets.Layout(width='150px', height='45px'),
            disabled=True
        )

        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Tiến độ:',
            style={'bar_color': '#4CAF50'}
        )

        self.status_label = widgets.Label(
            value="🔄 Đang khởi tạo hệ thống...",
            style={'font_weight': 'bold'}
        )

        self.btn_start.on_click(self._start_processing)
        self.btn_stop.on_click(self._stop_processing)

        display(widgets.VBox([
            widgets.HBox([self.btn_start, self.btn_stop]),
            self.progress,
            self.status_label
        ]))

    def update_status(self, message, progress=None):
        """Cập nhật trạng thái"""
        self.status_label.value = message
        if progress is not None:
            self.progress.value = progress
        print(f"[{time.strftime('%H:%M:%S')}] {message}")

    def _start_processing(self, b):
        """Bắt đầu xử lý frame"""
        if self.vit_ready:
            self.stream.start()
            threading.Thread(target=self.process_frames, daemon=True).start()
            self.update_status("📹 Đang xử lý video...", 50)
        else:
            self.update_status("⚠ Vui lòng chờ model tải xong!", 0)

    def _stop_processing(self, b):
        """Dừng xử lý"""
        self.stream.stop()
        self.update_status("🛑 Đã dừng xử lý", 0)

    def load_vit_model(self):
        """Load ViT model từ file đã train"""
        try:
            self.update_status("⚡ Đang tải ViT model...", 30)

            if not os.path.exists(VIT_MODEL_PATH):
                raise FileNotFoundError(f"Không tìm thấy file {VIT_MODEL_PATH}")

            config = ViTConfig.from_pretrained('google/vit-base-patch16-224')
            config.num_labels = len(CLASS_NAMES)

            self.vit_model = ViTForImageClassification.from_pretrained(
                'google/vit-base-patch16-224',
                config=config,
                ignore_mismatched_sizes=True
            ).to(self.device)

            state_dict = torch.load(VIT_MODEL_PATH, map_location=self.device)

            if isinstance(state_dict, dict) and 'state_dict' in state_dict:
                self.vit_model.load_state_dict(state_dict['state_dict'])
            else:
                self.vit_model.load_state_dict(state_dict)

            self.vit_model.eval()

            self.processor = {
                'mean': [0.5, 0.5, 0.5],
                'std': [0.5, 0.5, 0.5],
                'size': {'height': 224, 'width': 224}
            }

            test_input = torch.randn(1, 3, 224, 224).to(self.device)
            with torch.no_grad():
                output = self.vit_model(test_input).logits
            print(f"✅ Kiểm tra model thành công! Output shape: {output.shape}")

            self.vit_ready = True
            self.update_status("🎯 Model đã sẵn sàng!", 100)
            self.btn_start.disabled = False
            self.btn_stop.disabled = False

        except Exception as e:
            self.handle_error(e)

    def handle_error(self, error):
        """Xử lý lỗi"""
        error_msg = str(error)[:200]
        self.update_status(f"❌ Lỗi: {error_msg}", 0)
        self.progress.bar_style = 'danger'
        traceback.print_exc()

    def process_frames(self):
        """Xử lý từng frame"""
        while self.stream.is_active:
            try:
                with self.stream.lock:
                    if self.stream.frame is None or self.stream.frame.size == 0:
                        time.sleep(0.01)
                        continue

                    frame = self.stream.frame.copy()

                # Tính FPS
                now = time.time()
                self.fps = 0.9 * self.fps + 0.1 / (now - self.last_time) if (now - self.last_time) > 0 else 0
                self.last_time = now

                # Phát hiện người
                boxes = self.detect_people(frame)
                self.results = []

                # Xử lý từng người
                for (x, y, w, h) in boxes[:1]:
                    try:
                        # Nhận diện khung xương
                        keypoints = self.get_pose(frame, (x, y, w, h))
                        if np.mean(keypoints[:, 2]) > 0.2:
                            roi = frame[y:y+h, x:x+w]
                            if roi.size > 0:
                                action, conf = self.predict_action(Image.fromarray(roi))
                                self.results.append({
                                    'bbox': (x, y, w, h),
                                    'action': action,
                                    'confidence': conf,
                                    'keypoints': keypoints
                                })
                    except Exception as e:
                        print(f"⚠ Lỗi xử lý người: {e}")
                        continue

                # Hiển thị kết quả
                self.display_results(frame)

            except Exception as e:
                print(f"⚠ Lỗi xử lý frame: {e}")
                time.sleep(0.001)

    def detect_people(self, image):
        """Phát hiện người dùng YOLO"""
        try:
            (H, W) = image.shape[:2]
            blob = cv2.dnn.blobFromImage(image, 1/255.0, (320, 320), swapRB=True, crop=False)

            net, ln = self.models['yolo']
            net.setInput(blob)
            outputs = net.forward(ln)

            boxes = []
            confidences = []

            for output in outputs:
                for detection in output:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]

                    if class_id == 0 and confidence > 0.5:
                        box = detection[0:4] * np.array([W, H, W, H])
                        (cx, cy, width, height) = box.astype("int")
                        x = int(cx - (width / 2))
                        y = int(cy - (height / 2))
                        boxes.append([x, y, int(width), int(height)])
                        confidences.append(float(confidence))

            indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            return [boxes[i] for i in indices] if len(indices) > 0 else []

        except Exception as e:
            print(f"⚠ Lỗi phát hiện người: {e}")
            return []

    def get_pose(self, image, bbox):
        """Nhận diện khung xương"""
        try:
            x, y, w, h = [int(v) for v in bbox]
            roi = image[y:y+h, x:x+w]

            if roi.size == 0:
                return np.zeros((17, 3))

            input_img = cv2.resize(roi, (192, 192))
            input_img = np.expand_dims(input_img, axis=0).astype(np.uint8)

            interpreter = self.models['movenet']
            interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_img)
            interpreter.invoke()
            keypoints = interpreter.get_tensor(interpreter.get_output_details()[0]['index']).copy()[0,0]

            keypoints[:, 0] = x + keypoints[:, 0] * w
            keypoints[:, 1] = y + keypoints[:, 1] * h

            return keypoints

        except Exception as e:
            print(f"⚠ Lỗi khung xương: {e}")
            return np.zeros((17, 3))

    def predict_action(self, image):
        """Dự đoán hành động"""
        try:
            # Tiền xử lý ảnh
            img = image.resize((224, 224))
            img_array = np.array(img) / 255.0
            img_array = (img_array - np.array(self.processor['mean'])) / np.array(self.processor['std'])
            img_tensor = torch.tensor(img_array, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(self.device)

            # Dự đoán
            with torch.no_grad():
                outputs = self.vit_model(img_tensor).logits
                probs = F.softmax(outputs, dim=1)
                conf, pred = torch.max(probs, 1)

            return CLASS_NAMES[pred.item()], conf.item()

        except Exception as e:
            print(f"⚠ Lỗi dự đoán: {e}")
            return "unknown", 0.0

    def display_results(self, image):
        """Hiển thị kết quả real-time với các điểm khớp chính xác"""
        try:
            output = image.copy()

            # Vẽ FPS
            cv2.putText(output, f"FPS: {self.fps:.1f}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # Vẽ từng người
            for person in self.results:
                x, y, w, h = person['bbox']
                keypoints = person['keypoints']

                # Bounding box
                cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Vẽ khung xương với màu sắc khác nhau cho từng phần
                for (i, j) in SKELETON_CONNECTIONS:
                    if (i < len(keypoints)) and (j < len(keypoints)) and (keypoints[i][2] > 0.2) and (keypoints[j][2] > 0.2):
                        # Phân loại màu theo loại khớp nối
                        if i in [5,6,0,1,2]:  # Đầu
                            color = (255, 192, 203)  # Màu hồng
                        elif i in [7,8,9,10]:  # Tay
                            color = (0, 255, 255)  # Màu vàng
                        elif i in [11,12,13,14,15,16]:  # Chân
                            color = (255, 0, 255)  # Màu tím
                        else:  # Thân
                            color = (0, 255, 0)  # Màu xanh lá

                        cv2.line(output,
                                (int(keypoints[i][0]), int(keypoints[i][1])),
                                (int(keypoints[j][0]), int(keypoints[j][1])),
                                color, 3)

                # Vẽ điểm khớp
                for i, kp in enumerate(keypoints):
                    if kp[2] > 0.2:
                        # Màu sắc khác nhau cho các loại khớp
                        if i in [5,6,0,1,2]:  # Đầu
                            color = (255, 0, 0)  # Đỏ
                        elif i in [7,8,9,10]:  # Tay
                            color = (0, 0, 255)  # Xanh dương
                        elif i in [11,12,13,14,15,16]:  # Chân
                            color = (0, 255, 0)  # Xanh lá
                        else:  # Thân
                            color = (255, 255, 0)  # Vàng

                        cv2.circle(output, (int(kp[0]), int(kp[1])), 6, color, -1)

                # Nhãn hành động
                label = f"{person['action']} ({person['confidence']:.0%})"
                cv2.putText(output, label, (x, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            # Hiển thị real-time
            if self.stream.output_handle and time.time() - self.stream.last_update > 0.033:
                self.stream.output_handle.update(Image.fromarray(output))
                self.stream.last_update = time.time()

        except Exception as e:
            print(f"⚠ Lỗi hiển thị: {e}")

    def update_frame(self, frame_data):
        """Cập nhật frame từ JS"""
        try:
            if not frame_data or ',' not in frame_data:
                return

            # Kiểm tra header base64
            header, encoded = frame_data.split(',', 1)
            if 'image' not in header:
                return

            img_data = base64.b64decode(encoded)
            if len(img_data) == 0:
                return

            img_array = np.frombuffer(img_data, np.uint8)
            if img_array.size == 0:
                return

            frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
            if frame is not None:
                with self.stream.lock:
                    self.stream.frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        except Exception as e:
            pass

# ==================== CHẠY ỨNG DỤNG ====================
if __name__ == "__main__":
    # Xóa cache
    torch.cuda.empty_cache()

    # Kiểm tra hệ thống
    print("="*50)
    print(f"PyTorch {torch.__version__}")
    print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Không có'}")
    print("="*50)

    # Khởi chạy
    from google.colab import output
    app = ActionRecognitionApp()
    output.register_callback('notebook.processFrame', app.update_frame)

    print("""
    🚀 ỨNG DỤNG NHẬN DIỆN HÀNH ĐỘNG
    HƯỚNG DẪN:
    1. Chờ thanh tiến trình đạt 100%
    2. Nhấn [BẬT CAMERA] để bắt đầu
    3. Nhấn [TẮT CAMERA] để dừng
    """)

PyTorch 2.6.0+cu124
GPU: Tesla T4
⚙️ Thiết bị: Tesla T4
🔧 Đang khởi tạo model...
✅ YOLO loaded
✅ MoveNet loaded


[19:34:11] ⚡ Đang tải ViT model...


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Kiểm tra model thành công! Output shape: torch.Size([1, 15])
[19:34:13] 🎯 Model đã sẵn sàng!

    🚀 ỨNG DỤNG NHẬN DIỆN HÀNH ĐỘNG
    HƯỚNG DẪN:
    1. Chờ thanh tiến trình đạt 100%
    2. Nhấn [BẬT CAMERA] để bắt đầu
    3. Nhấn [TẮT CAMERA] để dừng
    


📷 Camera đã bật (640x480)


<IPython.core.display.Javascript object>

[19:34:14] 📹 Đang xử lý video...


<IPython.core.display.Javascript object>

🛑 Camera đã tắt
[19:38:22] 🛑 Đã dừng xử lý
